# Quoted Printable
メールの本文を変換するときにたまに使われるエンコード方式  
*8bitの文字を7biyのデータとして送ることができる。*

e.g.  
お腹が空いた => =E3=81=8A=E8=85=B9=E3=81=8C=E7=A9=BA=E3=81=84=E3=81=9F

## なぜメール7bitなのか
英語圏内のやりとりでよく使われるASCIIコードが7bitコードだから。コンピュータの黎明期で1bitが貴重な時代に造られたASCIIはわざわざ7bitコード体系にしてある。  
その名残で、メールのやりとりでは7bitが使われることがしばしばある。  
日本語などは7bit形式に一度エンコードしてから送る必要があると言うこと！

  
よって、元データがAscii文字を多く含む場合、エンコードされた文字列をほぼそのまま読むことができ、エンコード後のデータサイズも小さくなる。  
また、元データにAscii文字を含まない日本語などの場合は、エンコード後の文字列はほぼ意味をなさない文字列になることに加え、データサイズも非常に大きくなってしまいます。



## アルゴリズムの詳細
QuotedEncodingは、メールのエンコーディングに使われるため Content-Transfer-Encoding(MIME) として定義されています。  
  

1. 任意の1バイト(8ビット)は、"=" の後に続く2桁の16進数で表します。16進数は大文字を使います。  
    - 例: =3D  
2. 0x21("!")から0x3C("<")まで、および 0x3E(">")から 0x7E("~") は符号化せずそのままにする。  
    - 0x3D("=") は符号化します。  
3. 0x09(タブ文字) と 0x20(スペース) は符号化しません。ただし改行直前に来る場合は符号化します。  
4. 改行は CRLF を使います。テキストの場合、CR(0x0D)とLF(0x0A)は符号化しません。
5. エンコードされたテキストは1行あたり76文字以下でないといけません。76文字を超える行はソフト改行(=CRLF)を入れます。  
    - ソフト改行とは元データにとって無意味な改行、つまりエンコードの仕様上挟まれる改行のことです。
    - =と改行(CRLF)の間のスペースとタブはデコード時に無視されます。
    - 改行直前の意味のあるタブとスペースは 3. にある通り、符号化しておく必要があります。


In [79]:
# python の quopri ライブラリと同じ動きをする QPencodeingの実装
import quopri
import re

def QPeoncode(text):
    bytes_text = text.encode()
    character_count = 0
    result = []

    for i in range(len(bytes_text)):
        octet = bytes_text[i]
        is_converting = False

        if octet == 0x09 or octet == 0x20: # タブ or スペース ※16進数の20は10進数の32
            # 次に改行が続く場合、もしくは最後の文字の場合は変換。それ以外はそのまま出力
            if i < len(bytes_text)-2 and bytes_text[i+1] == 0x0D and bytes_text[i+2] == 0x0A:
                is_converting = True
            elif i < len(bytes_text)-2 and bytes_text[i+1] == 0x0A:
                is_converting = True
            elif i == len(bytes_text)-1:
                is_converting = True
            else:
                is_converting = False

        elif octet == 0x0A or octet == 0x0D: # LF or CR
            # 改行コードの類は符号化しない
            is_converting = False
        else:
            # 0x21("!")から0x3C("<")まで、および 0x3E(">")から 0x7E("~") は符号化せずそのままにする 0x3D("=") は符号化する
            is_converting = octet <= 0x20 or 0x7F <= octet or octet == 0x3D


        if is_converting:
            result.append("=%X" % (octet))
            character_count += 3
        else:
            result.append(chr(octet))
            character_count += 1


    return "".join(result)




def QPdecode(text):
    byte = bytes()
    # ソフト改行コードを全て削除
    text = re.sub('[\r\n]+$', '', text)

    idx = 0
    while idx < len(text):
        c = text[idx]
        if c == "=":
            octetStr = text[idx+1: idx+3]
            octet = bytes.fromhex(octetStr)
            byte += octet
            idx += 2
        else:
            # エンコードされていないASCII文字の場合
            byte += c.encode()

        idx += 1

    result = byte.decode()
    return result



text = "こんにちは世界 hello world!"
encoded = QPeoncode(text)
decoded = QPdecode(encoded)

print(text)
print(encoded)
print(decoded)

# quopriと同じ結果を返すかのチェック
print(encoded == quopri.encodestring(text.encode("utf-8")).decode())

# print(encoded == quopri.encodestring(text).decode())

# encoded2 = quopri.encodestring("""お腹が空いたfafaf 
# こんにちはaffagtrjgoipjgoauhgraou  """.encode("utf-8"))
# print(encoded)
# print("--------------------------------")
# print(encoded2.decode())
# print(encoded == encoded2.decode())

こんにちは世界 hello world!
=E3=81=93=E3=82=93=E3=81=AB=E3=81=A1=E3=81=AF=E4=B8=96=E7=95=8C hello world!
こんにちは世界 hello world!
True


In [61]:
# アルゴフルを参考にした実装(python quopriとは少し動作が異なる)

def QPeoncode(text):
    bytes_text = text.encode()
    character_count = 0
    result = []

    for i in range(len(bytes_text)):
        octet = bytes_text[i]
        is_converting = False

        if octet == 0x09 or octet == 0x20: # タブ or スペース ※16進数の20は10進数の32
            # 次に改行(CRLF)が続く場合は変換。それ以外はそのまま出力
            if i < len(bytes_text)-2 and bytes_text[i+1] == 0x0D and bytes_text[i+2] == 0x0A:
                is_converting = True
            else:
                is_converting = False

        elif octet == 0x0A or octet == 0x0D: # LF or CR
            # 改行コードの類は符号化しない
            is_converting = False
        else:
            # 0x21("!")から0x3C("<")まで、および 0x3E(">")から 0x7E("~") は符号化せずそのままにする 0x3D("=") は符号化する
            is_converting = octet <= 0x20 or 0x7F <= octet or octet == 0x3D


        if is_converting:
            result.append("=%X" % (octet))
            character_count += 3
        else:
            result.append(chr(octet))
            character_count += 1

        if character_count > 73:
            result.append("=\r\n")

        if result[-1] == "=\r\n":
            character_count = 0

    return "".join(result)


def QPdecode(text):
    byte = bytes()
    # ソフト改行コードを全て削除
    text = re.sub('[\r\n]+$', '', text)

    idx = 0
    while idx < len(text):
        c = text[idx]
        if c == "=":
            octetStr = text[idx+1: idx+3]
            octet = bytes.fromhex(octetStr)
            byte += octet
            idx += 2
        else:
            # エンコードされていないASCII文字の場合
            byte += c.encode()

        idx += 1

    result = byte.decode()
    return result
    
encoded = QPeoncode("""お腹が空いたfafaf 
こんにちはaffagtrjgoipjgoauhgraou  """)
encoded2 = quopri.encodestring("""お腹が空いたfafaf 
こんにちはaffagtrjgoipjgoauhgraou  """.encode("utf-8"))
print(encoded)
print("--------------------------------")
print(encoded2.decode())
print(encoded == encoded2.decode())

=E3=81=8A=E8=85=B9=E3=81=8C=E7=A9=BA=E3=81=84=E3=81=9Ffafaf 
=E3=81=93=E3=82=
=93=E3=81=AB=E3=81=A1=E3=81=AFaffagtrjgoipjgoauhgraou  
--------------------------------
=E3=81=8A=E8=85=B9=E3=81=8C=E7=A9=BA=E3=81=84=E3=81=9Ffafaf=20
=E3=81=93=E3=82=93=E3=81=AB=E3=81=A1=E3=81=AFaffagtrjgoipjgoauhgraou =20
False
